In [1]:
dataset = 'titanic'
reduced = 2000 

In [2]:
import os
import re

import numpy as np
import pandas as pd
pd.set_option('max_columns', None)

import plotly.graph_objects as go

# Data Loading

In [3]:
if dataset == 'titanic':
    #https://www.kaggle.com/c/titanic/data
    tit_sub = pd.read_csv('./data/Titanic/gender_submission.csv')
    tit_train = pd.read_csv('./data/Titanic/train.csv')
    tit_test = pd.read_csv('./data/Titanic/test.csv')
    df_train_final = pd.read_pickle("./data/Titanic/df_train_final")
    df_test_final = pd.read_pickle("./data/Titanic/df_test_final")

    from sklearn.preprocessing import StandardScaler
    scaler_cols = ['Age', 'Fare', 'Name_Length', 'Family_Size', 'Ticket_Frequency', 'Fare_Family_Size', 'Fare_Cat_Pclass']
    std = StandardScaler()
    std.fit(df_train_final[scaler_cols])
    df_train_final.loc[:, scaler_cols] = std.transform(df_train_final[scaler_cols])
    df_test_final.loc[:, scaler_cols] = std.transform(df_test_final[scaler_cols])

    features = ['Pclass', 'Sex', 'Age', 'Fare', 'Title', 'Name_Length', 'Emb_C',
           'Emb_Q', 'Emb_S', 'Title_Master', 'Title_Miss', 'Title_Mr', 'Title_Mrs',
           'Title_Other', 'Title_Royal', 'Family_Size',
           'Family_Friends_Surv_Rate', 'Cabin_Clean',
           'Ticket_Frequency', 'Tkt_AS', 'Tkt_C', 'Tkt_CA',
           'Tkt_CASOTON', 'Tkt_FC', 'Tkt_FCC', 'Tkt_Fa', 'Tkt_LINE', 
           'Tkt_NUM', 'Tkt_PC', 'Tkt_PP', 'Tkt_PPP', 'Tkt_SC', 'Tkt_SCA',
           'Tkt_SCAH', 'Tkt_SCAHBasle', 'Tkt_SCOW', 'Tkt_SCPARIS', 'Tkt_SCParis',
           'Tkt_SOC', 'Tkt_SOP', 'Tkt_SOPP', 'Tkt_SOTONO', 'Tkt_SOTONOQ', 'Tkt_SP',
           'Tkt_STONO', 'Tkt_SWPP', 'Tkt_WC', 'Tkt_WEP', 'Fare_Cat', 'Child', 'Senior']

    features_train = ['Survived', 'Pclass', 'Sex', 'Age', 'Fare', 'Title', 'Name_Length', 'Emb_C',
           'Emb_Q', 'Emb_S', 'Title_Master', 'Title_Miss', 'Title_Mr', 'Title_Mrs',
           'Title_Other', 'Title_Royal', 'Family_Size',
           'Family_Friends_Surv_Rate', 'Cabin_Clean',
           'Ticket_Frequency', 'Tkt_AS', 'Tkt_C', 'Tkt_CA',
           'Tkt_CASOTON', 'Tkt_FC', 'Tkt_FCC', 'Tkt_Fa', 'Tkt_LINE',
           'Tkt_NUM', 'Tkt_PC', 'Tkt_PP', 'Tkt_PPP', 'Tkt_SC', 'Tkt_SCA',
           'Tkt_SCAH', 'Tkt_SCAHBasle', 'Tkt_SCOW', 'Tkt_SCPARIS', 'Tkt_SCParis',
           'Tkt_SOC', 'Tkt_SOP', 'Tkt_SOPP', 'Tkt_SOTONO', 'Tkt_SOTONOQ', 'Tkt_SP',
           'Tkt_STONO', 'Tkt_SWPP', 'Tkt_WC', 'Tkt_WEP', 'Fare_Cat', 'Child', 'Senior']

    df_train_final = df_train_final[features_train]
    df_test_final = df_test_final[features]

    # train/test/val split
    features = df_test_final.columns.to_list()
    X_train = df_train_final[features]
    Y_train = df_train_final['Survived']
    X_test = df_test_final.reset_index(drop=True)

    c=pd.read_csv('./data/Titanic/titanic_test_labels.csv')
    test_data_with_labels = c.copy()
    for i, name in enumerate(test_data_with_labels['name']):
        if '"' in name:
            test_data_with_labels['name'][i] = re.sub('"', '', name)
    for i, name in enumerate(tit_test['Name']):
        if '"' in name:
            tit_test['Name'][i] = re.sub('"', '', name)
    survived = []
    for name in tit_test['Name']:
        survived.append(int(test_data_with_labels.loc[test_data_with_labels['name'] == name]['survived'].values[-1]))
    Y_test = pd.Series(survived,index=X_test.index)

    Y_train = Y_train.to_numpy()
    Y_test = Y_test.to_numpy()
elif dataset == 'adult':
    train_df = pd.read_csv('./data/Adult/adult.data',
                           header=None,
                           names=['age','workclass','fnlwgt','education','education_num','marital_status','occupation',
                                  'relationship','race','sex','capital_gain','capital_loss','hours_per_week','native_country','target'])
    test_df = pd.read_csv('./data/Adult/adult.test',
                           header=0,
                           names=['age','workclass','fnlwgt','education','education_num','marital_status','occupation',
                                  'relationship','race','sex','capital_gain','capital_loss','hours_per_week','native_country','target'])

    train_df = train_df[np.sum(train_df.values==' ?',axis=1)==0]
    test_df = test_df[np.sum(test_df.values==' ?',axis=1)==0]

    train_df = train_df.drop(columns='education_num')
    test_df = test_df.drop(columns='education_num')
    
    from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder

    categorical_columns = ['workclass','education','marital_status','occupation','relationship','race','sex','native_country']

    enc = OneHotEncoder(handle_unknown='ignore')
    train_df[enc.get_feature_names()] = enc.fit_transform(train_df.loc[:,categorical_columns]).astype(int).toarray()
    test_df[enc.get_feature_names()] = enc.transform(test_df.loc[:,categorical_columns]).astype(int).toarray()

    ordinal_columns = ['target']

    enc = OrdinalEncoder()
    train_df.loc[:,ordinal_columns] = enc.fit_transform(train_df.loc[:,ordinal_columns].values).astype(int)
    test_df.loc[:,ordinal_columns] = enc.transform(test_df.loc[:,ordinal_columns].values).astype(int)

    train_df = train_df.drop(columns=categorical_columns)
    test_df = test_df.drop(columns=categorical_columns)
    
    from sklearn.preprocessing import MinMaxScaler
    scaler_cols = ['age', 'fnlwgt', 'capital_gain', 'capital_loss', 'hours_per_week']
    std = MinMaxScaler()
    std.fit(train_df.loc[:,scaler_cols])
    train_df.loc[:, scaler_cols] = std.transform(train_df.loc[:,scaler_cols])
    test_df.loc[:, scaler_cols] = std.transform(test_df.loc[:,scaler_cols])
    
    X_train = train_df.drop('target',axis=1)
    Y_train = train_df.loc[:,'target']
    X_test = test_df.drop('target',axis=1)
    Y_test = test_df.loc[:,'target']
    
    
else:
    raise Exception('dataset name not recognized, only adult or titanic are supported')

<ipython-input-3-28ea6db97c30>:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data_with_labels['name'][i] = re.sub('"', '', name)
<ipython-input-3-28ea6db97c30>:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tit_test['Name'][i] = re.sub('"', '', name)


In [4]:
X_train

,Pclass,Sex,Age,Fare,Title,Name_Length,Emb_C,Emb_Q,Emb_S,Title_Master,Title_Miss,Title_Mr,Title_Mrs,Title_Other,Title_Royal,Family_Size,Family_Friends_Surv_Rate,Cabin_Clean,Ticket_Frequency,Tkt_AS,Tkt_C,Tkt_CA,Tkt_CASOTON,Tkt_FC,Tkt_FCC,Tkt_Fa,Tkt_LINE,Tkt_NUM,Tkt_PC,Tkt_PP,Tkt_PPP,Tkt_SC,Tkt_SCA,Tkt_SCAH,Tkt_SCAHBasle,Tkt_SCOW,Tkt_SCPARIS,Tkt_SCParis,Tkt_SOC,Tkt_SOP,Tkt_SOPP,Tkt_SOTONO,Tkt_SOTONOQ,Tkt_SP,Tkt_STONO,Tkt_SWPP,Tkt_WC,Tkt_WEP,Fare_Cat,Child,Senior
0,3.0,0.0,-0.579342,-0.502445,1.0,-0.451760,0.0,0.0,1.0,0,0,1,0,0,0,0.059160,0.3838,1.0,-0.624404,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0,0
1,1.0,1.0,0.614433,0.786845,3.0,2.580638,1.0,0.0,0.0,0,0,0,1,0,0,0.059160,1.0000,7.0,-0.067503,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0,0
2,3.0,1.0,-0.280898,-0.488854,2.0,-0.451760,0.0,0.0,1.0,0,1,0,0,0,0,-0.560975,0.3838,1.0,-0.624404,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,0,0
3,1.0,1.0,0.390600,0.420730,3.0,1.657734,0.0,0.0,1.0,0,0,0,1,0,0,0.059160,0.5000,7.0,-0.067503,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0,0
4,3.0,0.0,0.390600,-0.486337,1.0,-0.319916,0.0,0.0,1.0,0,0,1,0,0,0,-0.560975,0.3838,1.0,-0.624404,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,2.0,0.0,-0.206287,-0.386671,4.0,-0.583603,0.0,0.0,1.0,0,0,0,0,1,0,-0.560975,0.3838,1.0,-0.624404,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0,0
887,1.0,1.0,-0.803174,-0.044381,2.0,0.207457,0.0,0.0,1.0,0,1,0,0,0,0,-0.560975,0.3838,8.0,-0.624404,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0,0
888,3.0,1.0,-0.033414,-0.176263,2.0,1.394048,0.0,0.0,1.0,0,1,0,0,0,0,1.299429,0.0000,1.0,1.046299,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,0,0
889,1.0,0.0,-0.280898,-0.044381,1.0,-0.715446,1.0,0.0,0.0,0,0,1,0,0,0,-0.560975,0.3838,7.0,-0.624404,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0,0


# Model

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F


if dataset == 'titanic':
    class FFNN_VAE(nn.Module):
        def __init__(self, input_shape, hidden, dropout_p=0.3, latent_dim=2):
            super(FFNN_VAE, self).__init__()

            # encoding components

            self.fc1 = nn.Linear(input_shape,hidden[0])
            self.drop1 = nn.Dropout(dropout_p)
            self.fc2 = nn.Linear(hidden[0],hidden[1])
            # Latent vectors mu and sigma
            self.fc3_mu = nn.Linear(hidden[1], latent_dim)      
            self.fc3_logvar = nn.Linear(hidden[1], latent_dim)  

            # Sampling vector
            self.fc4 = nn.Sequential(
                nn.Linear(latent_dim, hidden[1]),
                nn.Dropout(dropout_p),
                nn.ReLU(inplace=True)
            )
            # Decoder
            self.fc5 = nn.Sequential(
                nn.Linear(hidden[1], hidden[0]),
                nn.Dropout(dropout_p),
                nn.ReLU(inplace=True)
            )
            self.fc6 = nn.Linear(hidden[0], input_shape)

        def encode(self, x):
            x = F.relu(self.fc1(x))
            x = F.relu(self.fc2(self.drop1(x)))
            mu, logvar = self.fc3_mu(x), self.fc3_logvar(x)
            return mu, logvar

        def reparameterize(self, mu, logvar):
            if self.training:
                std = logvar.mul(0.5).exp_()
                eps = Variable(std.data.new(std.size()).normal_())
                return eps.mul(std).add_(mu)
            else:
                return mu

        def decode(self, z):
            x = self.fc4(z)
            x = self.fc5(x)
            x = self.fc6(x)
            return x

        def forward(self, x):
            mu, logvar = self.encode(x)
            z = self.reparameterize(mu, logvar)
            x_reconst = self.decode(z)
            return x_reconst, z, mu, logvar

    # hyperparameters
    latent_dim = 2    # latent dim extracted by 2D CNN
    dropout_p = 0       # dropout probability
    hidden = [24,12]

elif dataset == 'adult':
    class FFNN_VAE(nn.Module):
        def __init__(self, input_shape, hidden, latent_dim=2, dropout_p=0):
            super(FFNN_VAE, self).__init__()

            # encoding components

            self.fc1 = nn.Linear(input_shape, hidden[0])
            self.fc2 = nn.Linear(hidden[0], hidden[1])
            self.fc3 = nn.Linear(hidden[1], hidden[2])
            # Latent vectors mu and sigma
            self.fc4_mu = nn.Linear(hidden[2], latent_dim)      
            self.fc4_logvar = nn.Linear(hidden[2], latent_dim) 

            # Sampling vector
            self.fc5 = nn.Linear(latent_dim, hidden[2])
            # Decoder
            self.fc6 = nn.Linear(hidden[2], hidden[1])
            self.fc7 = nn.Linear(hidden[1], hidden[0])
            self.fc8 = nn.Linear(hidden[0], input_shape)

        def encode(self, x):
            x = F.relu(self.fc1(x))
            x = F.relu(self.fc2(x))
            x = F.relu(self.fc3(x))
            mu, logvar = self.fc4_mu(x), self.fc4_logvar(x)
            return mu, logvar

        def reparameterize(self, mu, logvar):
            if self.training:
                std = logvar.mul(0.5).exp_()
                eps = Variable(std.data.new(std.size()).normal_())
                return eps.mul(std).add_(mu)
            else:
                return mu

        def decode(self, z):
            x = F.relu(self.fc5(z))
            x = F.relu(self.fc6(x))
            x = F.relu(self.fc7(x))
            x = self.fc8(x)
            return x

        def forward(self, x):
            mu, logvar = self.encode(x)
            z = self.reparameterize(mu, logvar)
            x_reconst = self.decode(z)
            return x_reconst, z, mu, logvar

    # hyperparameters
    latent_dim = 2    
    hidden = [50,25,12]
    dropout_p = 0

# Create Model
FFNN_VAE_model = FFNN_VAE(len(X_train.columns),hidden = hidden, dropout_p=dropout_p,latent_dim=latent_dim)
print(FFNN_VAE_model)

if dataset == 'titanic':
    FFNN_VAE_model.load_state_dict(torch.load('./models/VAE_Titanic.pt'))
elif dataset == 'adult':
    FFNN_VAE_model.load_state_dict(torch.load('./models/VAE_Adult.pt'))

FFNN_VAE(
  (fc1): Linear(in_features=51, out_features=24, bias=True)
  (drop1): Dropout(p=0, inplace=False)
  (fc2): Linear(in_features=24, out_features=12, bias=True)
  (fc3_mu): Linear(in_features=12, out_features=2, bias=True)
  (fc3_logvar): Linear(in_features=12, out_features=2, bias=True)
  (fc4): Sequential(
    (0): Linear(in_features=2, out_features=12, bias=True)
    (1): Dropout(p=0, inplace=False)
    (2): ReLU(inplace=True)
  )
  (fc5): Sequential(
    (0): Linear(in_features=12, out_features=24, bias=True)
    (1): Dropout(p=0, inplace=False)
    (2): ReLU(inplace=True)
  )
  (fc6): Linear(in_features=24, out_features=51, bias=True)
)


In [6]:
with torch.no_grad():
    FFNN_VAE_model.eval()
    rec_x_train, z_train, mu, logvar = FFNN_VAE_model(torch.tensor(X_train.to_numpy(),dtype=torch.float32))
    rec_x_test, z_test, mu, logvar = FFNN_VAE_model(torch.tensor(X_test.to_numpy(),dtype=torch.float32))

In [7]:
if dataset == 'adult':
    reduced = np.random.permutation(range(15059))[:reduced]
    X_test = X_test.iloc[reduced]
    X_test.reset_index(drop=True)
    Y_test = Y_test.iloc[reduced]
    Y_test.reset_index(drop=True)
    z_test = z_test[reduced]

In [8]:
if dataset == 'titanic':
    x_min, x_max = z_test[:, 0].min(), z_test[:, 0].max()
    y_min, y_max = z_test[:, 1].min(), z_test[:, 1].max()
elif dataset == 'adult':
    x_min, x_max = z_test[:, 0].min(), z_test[:, 0].max()
    y_min, y_max = z_test[:, 1].min(), z_test[:, 1].max()

# APP

In [9]:
import plotly.express as px
import plotly.graph_objects as go

shap_fig = go.Figure()
clustering_fig = go.Figure()
violin_plot = go.Figure()

colorscale = [[0, '#3b4cc0'],[1, '#b40426']]

if dataset == 'titanic':
    target_name = 'dead'
elif dataset == 'adult':
    target_name = '<50K'

shap_fig.add_trace(go.Scatter(x=z_test[:, 0].numpy(), 
                              y=z_test[:, 1].numpy(),
                              mode='markers',
                              name=target_name,
                              marker=dict(
                                 size=10,
                                 symbol='circle',
                                 color=Y_test, 
                                 opacity=0.5,
                                 colorscale=colorscale,
                                 line=dict(width=1,
                                           color='Black'))
                             )
                  )

clustering_fig.add_trace(go.Scatter(x=z_test[:, 0].numpy(),
                                    y=z_test[:, 1].numpy(),
                                    mode='markers',
                                    name=target_name,
                                    marker=dict(
                                        size=10,
                                        symbol='circle',
                                        color=Y_test, 
                                        opacity=0.5,
                                        colorscale=colorscale,
                                        line=dict(width=1,
                                                  color='Black'))))

violin_plot.add_trace(go.Violin(name='left',
                      side='negative',
                      line_color='blue',
                      bandwidth=0.25,
                      points=False)
             )
violin_plot.add_trace(go.Violin(name='right',
                      side='positive',
                      line_color='orange',
                      bandwidth=0.25,
                      points=False)
             )
violin_plot.update_traces(meanline_visible=True)
violin_plot.update_layout(violingap=0, violinmode='overlay');

shap_fig.add_trace(go.Scatter(x=[0], y=[0] ,mode='markers',name='pointer',showlegend=False,
                         marker=dict(
                             size=30,
                             symbol='x',
                             color='#000000')))

shap_fig.add_trace(go.Scatter(x=[0], y=[0] ,mode='markers',name='expected_value',showlegend=False,
                         marker=dict(
                             size=35,
                             symbol='triangle-up',
                             color='#a834e7')))

names = ['vector_0','vector_1','vector_2','vector_3','vector_4','vector_5','vector_6','vector_7','vector_8','vector_9']

annotations = [
    go.layout.Annotation(
            #start
            x=0, 
            y=0,
            xref="x",
            yref="y",
            #end
            ax=0, 
            ay=0,
            axref = "x", 
            ayref = "y",
            showarrow=True,
            arrowside='start',
            arrowhead=2, # type od head [0,8]
            arrowsize=1, # head dimension 
            arrowwidth=2, # arrow dimension
            arrowcolor="#000000",
            name=name,
            text='  ',
            hovertext='baseline',
            font=dict(
                family="Courier New, monospace",
                size=1,
                color="#ffffff"
            ),
            bgcolor='#ff7f0e'
            ) for name in names]+[
     go.layout.Annotation(
            #start
            x=0, 
            y=0,
            xref="x",
            yref="y",
            #end
            ax=0, 
            ay=0,
            axref = "x", 
            ayref = "y",
            showarrow=True,
            arrowside='start',
            arrowhead=2, # type od head [0,8]
            arrowsize=1, # head dimension 
            arrowwidth=2, # arrow dimension
            arrowcolor="#000000",
            name='others_contrib',
            text='  ',
            hovertext='other contributions',
            font=dict(
                family="Courier New, monospace",
                size=1,
                color="#ffffff"
            ),
            bgcolor='#ff7f0e'
            )
]

clustering_fig.update_layout(
    dragmode='lasso',
    showlegend=True,
    autosize=True,
    margin=dict(
    l=5,
    r=0,
    b=5,
    t=10,
    pad=0
))

clustering_fig.update_xaxes(
    range=[x_min, x_max],  
)

clustering_fig.update_yaxes(
    scaleanchor = "x",
    scaleratio = 1,
  )

shap_fig.update_layout(
    showlegend=True,
    annotations=annotations,
    autosize=True,
    #height=700,
    #width=700,
    margin=dict(
    l=5,
    r=0,
    b=5,
    t=10,
    pad=0
))

shap_fig.update_xaxes(
    range=[x_min, x_max],  
)

shap_fig.update_yaxes(
    range=[y_min, y_max],
    scaleanchor = "x",
    scaleratio = 1,
  );

In [10]:
slider_names = ['slider_name_0','slider_name_1','slider_name_2','slider_name_3','slider_name_4',
                'slider_name_5','slider_name_6','slider_name_7','slider_name_8','slider_name_9']

slider_figs = [go.Figure() for name in slider_names]

slider_annotations = [[
    go.layout.Annotation(
            #start
            x=0, 
            y=0,
            xref="x",
            yref="y",
            #end
            ax=1, 
            ay=1,
            axref = "x", 
            ayref = "y",
            showarrow=True,
            arrowside='start',
            arrowhead=2, # type od head [0,8]
            arrowsize=1, # head dimension 
            arrowwidth=2, # arrow dimension
            arrowcolor="#994F00",
            name=name+'_top',
            ),
    go.layout.Annotation(
            #start
            x=0, 
            y=0,
            xref="x",
            yref="y",
            #end
            ax=-1, 
            ay=-1,
            axref = "x", 
            ayref = "y",
            showarrow=True,
            arrowside='start',
            arrowhead=2, # type od head [0,8]
            arrowsize=1, # head dimension 
            arrowwidth=2, # arrow dimension
            arrowcolor="#0C7BDC",
            name=name+'_bottom',
            )
    ] for name in slider_names] 

for i in range(len(slider_names)):
    slider_figs[i].update_layout(
        height = 50,
        width = 50,
        annotations = slider_annotations[i],
        margin=go.layout.Margin(
            l=0, #left margin
            r=0, #right margin
            b=0, #bottom margin
            t=0  #top margin
        ),
        plot_bgcolor = '#f2f2f2'
    )

    #slider_figs[i].add_trace(go.Scatter(x=[0], y=[0] , mode='markers', name='expected_value_'+slider_names[i],showlegend=False,
    #                         marker=dict(
    #                             size=10,
    #                             symbol='triangle-up',
    #                             color='#a834e7'),
    #                          hoverinfo='none'))
    
    slider_figs[i].update_xaxes(
        showgrid = False, 
        zeroline = False, 
        visible  = False,
    )
    
    slider_figs[i].update_yaxes(
        showgrid = False, 
        zeroline = False, 
        visible  = False,
    )
    
    slider_figs[i].update_xaxes(
    range=[-1, 1],  
    )
    
    slider_figs[i].update_yaxes(
    range=[-1, 1],
    scaleanchor = "x",
    scaleratio = 1
    )

#slider_figs[0].show(config = {'displayModeBar': False}) #set this into dcc.graph 

In [11]:
import shap

def wrapper(X):
    FFNN_VAE_model.eval()
    rec_x_train, z, mu, logvar = FFNN_VAE_model(torch.tensor(X).float())
    return z.detach().numpy()


# use Kernel SHAP to explain test set predictions
explainer = shap.KernelExplainer(wrapper, shap.sample(X_train,100), link="identity")

#    data : numpy.array or pandas.DataFrame or shap.common.DenseData or any scipy.sparse matrix
#        The background dataset to use for integrating out features. To determine the impact
#        of a feature, that feature is set to "missing" and the change in the model output
#        is observed. Since most models aren't designed to handle arbitrary missing data at test
#        time, we simulate "missing" by replacing the feature with the values it takes in the
#        background dataset. So if the background dataset is a simple sample of all zeros, then
#        we would approximate a feature being missing by setting it to zero. For small problems
#        this background dataset can be the whole training set, but for larger problems consider
#        using a single reference value or using the kmeans function to summarize the dataset.
#        Note: for sparse case we accept any sparse matrix but convert to lil format for
#        performance.
#    link : "identity" or "logit"
#        A generalized linear model link to connect the feature importance values to the model
#        output. Since the feature importance values, phi, sum up to the model output, it often makes
#        sense to connect them to the output with a link function where link(output) = sum(phi).
#        If the model output is a probability then the LogitLink link function makes the feature
#        importance values have log-odds units. 

In [12]:
mins = np.min(X_train.values,axis=0)
mins = np.round(mins,2)
maxs = np.max(X_train.values,axis=0)
maxs = np.round(maxs,2)
selected_point = 0
change_point = 0
query = X_test.values[selected_point,:]
exp_input_values = X_test.mean(axis=0).values

# main function to output the xai scores
def compute_XAI_values(query):
    
    #original_shap
    shap_values = explainer.shap_values(query, nsamples='auto')
    return explainer.expected_value[0], shap_values[0] ,explainer.expected_value[1], shap_values[1]
    

exp_x, xai_x, exp_y, xai_y = compute_XAI_values(query)
shap_fig.for_each_trace(
        lambda trace: trace.update(x=[exp_x], y=[exp_y]) if trace.name == "expected_value" else (),
        )

s = [exp_x, exp_y]
#select only the 10 most important features
indices = np.argsort(np.sqrt(xai_x**2+xai_y**2))[-10:][::-1]

for i in range(len(slider_names)):
    vec = np.array([xai_x[indices][i],xai_y[indices][i]])
    vec /= np.linalg.norm(vec)
    if query[indices][i]>exp_input_values[indices][i]:
        slider_annotations[i][0]['ax'] = vec[0]
        slider_annotations[i][0]['ay'] = vec[1]
        slider_annotations[i][1]['ax'] = -vec[0]
        slider_annotations[i][1]['ay'] = -vec[1]
    else:
        slider_annotations[i][0]['ax'] = -vec[0]
        slider_annotations[i][0]['ay'] = -vec[1]
        slider_annotations[i][1]['ax'] = vec[0]
        slider_annotations[i][1]['ay'] = vec[1]
    slider_figs[i].update_layout(annotations = slider_annotations[i])
        
# list of the increments of the sliders
if dataset == 'titanic':
    steps = [1,1,0.01,0.01,1,0.01,1,1,1,1,1,1,1,1,1,0.01,0.01,1,0.01,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1]
elif dataset == 'adult':
    steps=[0.1]*5+[1]*98

In [13]:
import json
import dash
from jupyter_dash import JupyterDash
#import dash_core_components as dcc
from dash import dcc
#import dash_html_components as html
from dash import html
from dash.dependencies import Input, Output, State
from dash.exceptions import PreventUpdate

# function to select the ticks of the sliders
def select_ticks(m, M, x, e):
    # round values
    if x % 1:
        x = np.round(x,2)
    else: x = int(x)
    if m % 1:
        m = np.round(m,2)
    else: m = int(m)
    if M % 1:
        M = np.round(M,2)
    else: M = int(M)
    if e % 1:
        e = np.round(e,2)
    else: e = int(e)
    if x==m:
        ticks = dict(sorted({e:{'label':'e','style': {'color': '800081','font-size':'1rem'}},
                    x:{'label':'x','style': {'color': '#994F00','font-size':'1rem'}},
                    M:{'label':str(M),'style': {'color': '#0C7BDC','font-size':'1rem'}}
                    }.items()))
    elif x==M:
        ticks = dict(sorted({m:{'label':str(m), 'style': {'color': '#994F00','font-size':'1rem'}},
                    e:{'label':'e','style': {'color': '#800081','font-size':'1rem'}},
                    x:{'label':'x','style': {'color': '#0C7BDC','font-size':'1rem'}}
                    }.items()))
    else:
        ticks = dict(sorted({m:{'label':str(m), 'style': {'color': '#994F00','font-size':'1rem'}},
                    e:{'label':'e','style': {'color': '#800081','font-size':'1rem'}},
                    x:{'label':'x','style': {'color': '#000000','font-size':'1rem'}},
                    M:{'label':str(M),'style': {'color': '#0C7BDC','font-size':'1rem'}}
                    }.items()))
    return ticks

app = JupyterDash(__name__,meta_tags=[{"name": "viewport", "content": "width=device-width"}])

app.layout = \
html.Div([
    html.Div([
        html.H3('Latent Space Exploration'),
        ],
        id='title',
        className='title'),
    html.Div([
        html.Div([
            html.Div(
                # title above the sliders
                [html.Div([
                        html.Div([
                            html.H5('Name')
                        ],
                        style={'width':'35%','text-decoration': 'underline'}),
                        html.Div([
                            html.H5('Slider')
                        ],
                        style={'width':'45%',
                               'text-decoration': 'underline'}),
                        html.Div([
                            html.H5('Value')
                        ],
                        style={'width':'10%','text-decoration': 'underline','padding-left': '5px'}),
                        html.Div([
                            html.H5('Vector')
                        ],
                        style={'width':'10%','text-decoration': 'underline','padding-left': '5px'})
                    ],
                id = 'index',
                className = 'index')]+[
                # cycle for for the 10 sliders
                 html.Div([
                    html.Div([
                        html.H5(str(X_train.columns[indices[i]])+' : ', id = 'slider_variable_name_'+str(i))
                        ],
                        className='variable_name'),
                    dcc.Slider(
                        min = mins[indices[i]],
                        max = maxs[indices[i]],
                        value = query[indices[i]],
                        step = steps[indices[i]],
                        marks = select_ticks(mins[indices[i]], maxs[indices[i]], query[indices[i]], exp_input_values[indices[i]]),
                        id = 'slider_'+str(i),
                        className = 'slider'),
                    html.Div(id = 'slider_'+str(i)+'_value', className = 'slider_value'),
                    html.Div([
                        dcc.Graph(
                            id = 'slider_graph_'+str(i),
                            figure = slider_figs[i],
                            config = {'displayModeBar': False})],
                        id = 'slider_'+str(i)+'_vector', 
                        className = 'slider_value'),
                    ],
                    id = 'variable_'+str(i),
                    className = 'variable') for i in range(10)],
                id='inputs',
                className='inputs'),
            html.Div([
                dcc.Graph(
                    id='space-plot',
                    figure=shap_fig,
                    #config = {'displayModeBar': False}
                )
                ],
                id='graph',
                className='graph'),
            ],
            id='shap_page',
            className='shap_page'),
        html.Div([
            html.Div([
                html.Div([
                    dcc.Graph(
                        id='left_plot',
                        figure=clustering_fig),
                    ],
                    className='left_plot'),
                html.Div([
                    dcc.Graph(
                        id='right_plot',
                        figure=clustering_fig)
                    ],
                    className='right_plot')
                ],
                id='graphs',
                className='graphs'),
            html.Div([
                dcc.Graph(
                    id='violin_plot',
                    figure=violin_plot)
            ],
            className='violin-graph')
        ],
        id='clustering_page',
        className='clustering_page')
        ],
        id='main_body',
        className='main_body'
        )
    ])

#function to update the plot and the values to the 10 most important features
@app.callback(
    #Output 3 value for slider + the plot
    [dash.dependencies.Output('slider_'+str(i)+'_value', 'children') for i in range(10)]+
    [dash.dependencies.Output('space-plot', 'figure')],
    # As input we only have the 10 values from the sliders
    [dash.dependencies.Input('slider_'+str(i), 'value') for i in range(10)],
    dash.dependencies.State('space-plot', 'relayoutData'))
def update_output(value0,value1,value2,value3,value4,value5,value6,value7,value8,value9,fig_data):
    v = [value0,value1,value2,value3,value4,value5,value6,value7,value8,value9]
    # query is the value to explain 
    # in this for loop change the value of the query according to the slider values
    query = X_test.values[selected_point,:]
    # Problem when selecting a new point: the slider values changes one at a time and this function is triggered 10 times!!! one for every changes with the older values
    #global change_point
    #if change_point:
    #    if dash.callback_context.triggered[0]['prop_id'] != 'slider_9.value':
    #        raise PreventUpdate
    #    else: change_point = 0
    for i in range(len(v)):
        query[indices[i]] = v[i]
    # compute xai values
    exp_x, xai_x, exp_y, xai_y = compute_XAI_values(query)
    
    #compute new z position
    with torch.no_grad():
        z , _ = FFNN_VAE_model.encode(torch.tensor(query).float())
        z = z.numpy()
    #update the scatter plot and the vector explanations
    shap_fig.for_each_trace(
        lambda trace: trace.update(x=[z[0]], y=[z[1]]) if trace.name == "pointer" else (),
        )
    s = [exp_x, exp_y]
    for i in range(len(v)):
        #start
        annotations[i]['x'] = s[0]
        annotations[i]['y'] = s[1]
        #end
        s[0] += xai_x[indices[i]]
        s[1] += xai_y[indices[i]]
        annotations[i]['ax'] = s[0]
        annotations[i]['ay'] = s[1]
        annotations[i]['hovertext'] = str(X_train.columns[indices[i]])
    annotations[-1]['x'] = s[0]
    annotations[-1]['y'] = s[1]
    annotations[-1]['ax'] = z[0]
    annotations[-1]['ay'] = z[1]
    # update with zoom level as selected by the user
    try:
        shap_fig.update_layout({'xaxis':{'range':[fig_data['xaxis.range[0]'],fig_data['xaxis.range[1]']]},
                           'yaxis':{'range':[fig_data['yaxis.range[0]'],fig_data['yaxis.range[1]']]},
                          },
                          annotations = annotations)
    except: shap_fig.update_layout(annotations = annotations)
    # the output must be a list [slider_value]*10+[fig]
    #v = np.array(v)
    #xai_x = xai_x[indices]
    #xai_y = xai_y[indices]
    out = v
    #out = list(np.vstack(v).transpose().ravel())
    # round the numbers
    for i in range(len(out)):
        if out[i]%1:
            out[i] = np.round(out[i],2)
        else: out[i] = int(out[i])
    return out + [shap_fig]
    
# select the point
@app.callback(
    sum([[dash.dependencies.Output('slider_variable_name_'+str(i), 'children'),
      dash.dependencies.Output('slider_'+str(i), 'min'),
      dash.dependencies.Output('slider_'+str(i), 'max'),
      dash.dependencies.Output('slider_'+str(i), 'value'),
      dash.dependencies.Output('slider_'+str(i), 'step'),
      dash.dependencies.Output('slider_'+str(i), 'marks'),
     ] for i in range(10)],[])+
    [dash.dependencies.Output('slider_graph_'+str(i), 'figure') for i in range(10)],
    dash.dependencies.Input('space-plot', 'clickData'), prevent_initial_call=True
    )
def display_click_data(clickData):
    global selected_point
    selected_point = clickData['points'][0]['pointIndex']
    global change_point
    change_point = 1
    query = X_test.values[selected_point,:]
    #print(selected_point)
    exp_x, xai_x, exp_y, xai_y = compute_XAI_values(query)
    global shap_fig
    shap_fig.for_each_trace(
        lambda trace: trace.update(x=[exp_x], y=[exp_y]) if trace.name == "expected_value" else (),
    )
    s = [exp_x, exp_y]
    #select only the 10 most important features
    global indices
    indices = np.argsort(np.sqrt(xai_x**2+xai_y**2))[-10:][::-1]
    
    for i in range(len(slider_names)):
        vec = np.array([xai_x[indices][i],xai_y[indices][i]])
        vec /= np.linalg.norm(vec)
        if query[indices][i]>exp_input_values[indices][i]:
            slider_annotations[i][0]['ax'] = vec[0]
            slider_annotations[i][0]['ay'] = vec[1]
            slider_annotations[i][1]['ax'] = -vec[0]
            slider_annotations[i][1]['ay'] = -vec[1]
        else:
            slider_annotations[i][0]['ax'] = -vec[0]
            slider_annotations[i][0]['ay'] = -vec[1]
            slider_annotations[i][1]['ax'] = vec[0]
            slider_annotations[i][1]['ay'] = vec[1]
        slider_figs[i].update_layout(annotations = slider_annotations[i])
    return sum([[str(X_train.columns[indices[i]])+' : ',
                 mins[indices[i]],
                 maxs[indices[i]],
                 query[indices[i]],
                 steps[indices[i]],
                 select_ticks(mins[indices[i]],
                              maxs[indices[i]],
                              query[indices[i]],
                              exp_input_values[indices[i]])
                ] for i in range(10)],[])+slider_figs

def update_violin_plot(idx_l,idx_r):
    global X_test
    x_l = X_test.iloc[idx_l==1,:].copy()
    x_r = X_test.iloc[idx_r==1,:].copy()
    ma = np.mean(x_l.to_numpy(),axis=0)
    mb = np.mean(x_r.to_numpy(),axis=0)
    indices = np.argsort(np.abs(ma-mb))[-15:]
    x_l = x_l.iloc[:,indices].melt()
    x_r = x_r.iloc[:,indices].melt()
    global violin_plot
    violin_plot.for_each_trace(
        lambda trace: trace.update(x=x_l['variable'], y=x_l['value']) if trace.name == "left" else (trace.update(x=x_r['variable'], y=x_r['value'])),
    )
    return violin_plot

@app.callback(
    dash.dependencies.Output('violin_plot', 'figure'),
    [dash.dependencies.Input('left_plot', 'selectedData'),
     dash.dependencies.Input('right_plot', 'selectedData')]
    )
def compute_violin(selectedData_left, selectedData_right):
    if selectedData_left:
        selected_points_left = [selectedData_left['points'][i]['pointIndex'] for i in range(len(selectedData_left['points']))]
        if selectedData_right:
            selected_points_right = [selectedData_right['points'][i]['pointIndex'] for i in range(len(selectedData_right['points']))]
            idx_l = np.zeros(len(X_test))
            idx_r = np.zeros(len(X_test))
            idx_r[selected_points_right]=1
            idx_l[selected_points_left]=1
            return update_violin_plot(idx_l,idx_r)
        else:
            idx_l = np.zeros(len(X_test))
            idx_l[selected_points_left]=1
            idx_r = np.ones(len(X_test))
            return update_violin_plot(idx_l,idx_r)
    else:
        if selectedData_right:
            selected_points_right = [selectedData_right['points'][i]['pointIndex'] for i in range(len(selectedData_right['points']))]
            idx_l = np.ones(len(X_test))
            idx_r = np.zeros(len(X_test))
            idx_r[selected_points_right]=1
            return update_violin_plot(idx_l,idx_r)
        else:
            violin_plot.for_each_trace(
                lambda trace: trace.update(x=X_test.melt()['variable'], y=X_test.melt()['value']) if trace.name == "right" else (trace.update(x=X_test.melt()['variable'], y=X_test.melt()['value'])),
            )
            return violin_plot

In [14]:
# run the app
app.run_server(mode='external',
               port=8090, 
               dev_tools_ui=True, 
               debug=True,
               dev_tools_hot_reload=True,
               threaded=True,
               #host='127.0.0.1:2595'
              )

OSError: Address 'http://127.0.0.1:8090' already in use.
    Try passing a different port to run_server.

Exception in thread Thread-4:
Traceback (most recent call last):
  File "/Users/francescobodria/anaconda3/envs/latent/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/Users/francescobodria/anaconda3/envs/latent/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/francescobodria/anaconda3/envs/latent/lib/python3.8/site-packages/retrying.py", line 49, in wrapped_f
    return Retrying(*dargs, **dkw).call(f, *args, **kw)
  File "/Users/francescobodria/anaconda3/envs/latent/lib/python3.8/site-packages/retrying.py", line 212, in call
    raise attempt.get()
  File "/Users/francescobodria/anaconda3/envs/latent/lib/python3.8/site-packages/retrying.py", line 247, in get
    six.reraise(self.value[0], self.value[1], self.value[2])
  File "/Users/francescobodria/anaconda3/envs/latent/lib/python3.8/site-packages/six.py", line 703, in reraise
    raise value
  File "/Users/francescobodria/anaconda3/envs/latent/